<a href="https://www.kaggle.com/code/aayushsin7a/musicrecommendersystem?scriptVersionId=199933755" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Music recommender system

### One of the most used machine learning algorithms is recommendation systems. A recommender (or recommendation) system (or engine) is a filtering system which aim is to predict a rating or preference a user would give to an item, eg. a film, a product, a song, etc.

### Which type of recommender can we have?

### There are two main types of recommender systems: 

- Content-based filters
- Collaborative filters
- Content-based filters predicts what a user likes based on what that particular user has liked in the past. On the other hand, collaborative-based filters predict what a user like based on what other users, that are similar to that particular user, have liked.

#### 1) Content-based filters
Recommendations done using content-based recommenders can be seen as a user-specific classification problem. This classifier learns the user's likes and dislikes from the features of the song.

The most straightforward approach is keyword matching.

In a few words, the idea behind is to extract meaningful keywords present in a song description a user likes, search for the keywords in other song descriptions to estimate similarities among them, and based on that, recommend those songs to the user.

#### How is this performed?

In our case, because we are working with text and words, Term Frequency-Inverse Document Frequency (TF-IDF) can be used for this matching process.

We'll go through the steps for generating a content-based music recommender system.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        



/kaggle/input/songsdata/songdata.csv


In [2]:
df=pd.read_csv("/kaggle/input/songsdata/songdata.csv")

In [3]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [4]:
df.head()

,artist,song,text
0,Rainbow,Spotlight Kid,You don't know what's happenin' you want to go...
1,Nazareth,Do You Think About It,What you gonna do if the fashion changes \nHa...
2,Phish,All Of These Dreams,There is a place on the mountain near by \nDe...
3,Carol Banawa,Maala Ala Mo Kaya,Maala-ala mo kaya \nAng sumpa mo sa akin \nN...
4,Pearl Jam,Future Days,"If I ever were to lose you, \nI'd surely lose..."


In [5]:
df.shape

(5000, 3)

- We can notice also the presence of \n in the text, so we are going to remove it.

In [6]:
df['text']=df['text'].str.replace(r'\n','')

- After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each song lyric, word-by-word. 

- Here, we pay particular attention to the arguments we can specify.

In [7]:
tfidf = TfidfVectorizer(analyzer='word',stop_words='english')

In [8]:
lyrics_matrix = tfidf.fit_transform(df['text'])

In [9]:
lyrics_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 267277 stored elements and shape (5000, 23908)>

'''
## How do we use this matrix for a recommendation?

- We now need to calculate the similarity of one lyric to another. We are going to use cosine similarity.

- We want to calculate the cosine similarity of each item with every other item in the dataset. So we just pass the lyrics_matrix as argument.
'''

In [10]:
cosine_similarities=cosine_similarity(lyrics_matrix)

- Once we get the similarities, we'll store in a dictionary the names of the 50 most similar songs for each song in our dataset.

In [11]:
cosine_similarities

array([[1.        , 0.00261621, 0.00529187, ..., 0.03948274, 0.07271885,
        0.00904653],
       [0.00261621, 1.        , 0.00110225, ..., 0.05365905, 0.0098965 ,
        0.00234905],
       [0.00529187, 0.00110225, 1.        , ..., 0.00189407, 0.01662245,
        0.00768165],
       ...,
       [0.03948274, 0.05365905, 0.00189407, ..., 1.        , 0.15891781,
        0.01581274],
       [0.07271885, 0.0098965 , 0.01662245, ..., 0.15891781, 1.        ,
        0.0322426 ],
       [0.00904653, 0.00234905, 0.00768165, ..., 0.01581274, 0.0322426 ,
        1.        ]])

In [12]:
similarities = {}

In [13]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[df['song'].iloc[i]] = [(cosine_similarities[i][x], df['song'][x], df['artist'][x]) for x in similar_indices][1:]

- After that, all the magic happens. We can use that similarity scores to access the most similar items and give a recommendation.

- For that, we'll define our Content based recommender class.

In [14]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [15]:
recommedations = ContentBasedRecommender(similarities)

In [16]:
recommendation = {
    "song": df['song'].iloc[10],
    "number_songs": 4 
}

In [17]:
recommedations.recommend(recommendation)

The 4 recommended songs for No Good Texas Rounder are:
Number 1:
Cowboy Bill by Garth Brooks with 0.243 similarity score
--------------------
Number 2:
As Good As New by ABBA with 0.2 similarity score
--------------------
Number 3:
Mama Said by Dusty Springfield with 0.18 similarity score
--------------------
Number 4:
Good To See You by Neil Young with 0.177 similarity score
--------------------


In [18]:
recommendation2 = {
    "song": df['song'].iloc[120],
    "number_songs": 4 
}

In [19]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Day In Day Out are:
Number 1:
Oo Ba La Baby by Eddie Cochran with 0.458 similarity score
--------------------
Number 2:
Unconditional Love by America with 0.404 similarity score
--------------------
Number 3:
Paradise by Bette Midler with 0.264 similarity score
--------------------
Number 4:
City Of The Angels by Journey with 0.214 similarity score
--------------------
